In [ ]:
#import library (tensorflow bc it says its good for image recognition meaning it can tell for differences in between images)
#https://www.nvidia.com/en-au/glossary/tensorflow/#:~:text=TensorFlow%20can%20be%20used%20to,such%20as%20partial%20differential%20equations.
import tensorflow as tf

#Keras (api we use) we can change if needed

#keras.models is where we get the type of model we use (Sequential)
#https://keras.io/api/models/
from tensorflow.keras.models import Sequential

#keras.layers is the type of layers
#https://keras.io/api/layers/   and use this link for details abt dense,flatten, and dropout    https://datascience.stackexchange.com/questions/44124/when-to-use-dense-conv1-2d-dropout-flatten-and-all-the-other-layers
from tensorflow.keras.layers import Dense, Flatten, Dropout

#below is a example model
#from tensorflow.keras.applications import ResNet50 (CNN model) bc it says it is a visual recognicition model when i searched it up. https://keras.io/api/applications/ (gives a list of models keras uses)
#https://wandb.ai/mostafaibrahim17/ml-articles/reports/The-Basics-of-ResNet50---Vmlldzo2NDkwNDE2#:~:text=ResNet50%20is%20a%20deep%20learning,model%20is%2050%20layers%20deep.
from tensorflow.keras.applications import ResNet50

#allows to rescale images and enhances different features over consecutive trials for the ai model(for it to tell differences)
#https://www.isahit.com/blog/what-is-the-purpose-of-image-preprocessing-in-deep-learning#:~:text=Even%20though%20geometric%20transformations%20of,features%20crucial%20for%20subsequent%20processing.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#chatgpt said to use this for some reason but I only see it allows us to stop our model during training.
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
from google.colab import files

In [ ]:
uploaded = files.upload()

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [ ]:
#self explainitory just replace path/to/____ with the file path

#need images
#also note that there should be two folders in the training images/test images (ai and human made images just name the folder depending on each one)
trainImages = Users/erich/Downloads/aiTrainingImages

#need images for this part
#note that there should NOT be two folders for testing data
testImages = Users/erich/Downloads/aiTestImages

NameError: name 'Users' is not defined

In [ ]:
#this is what allows us to change how the ai will see the image
#link below gives list of possible values although I didn't use all of them ex:brightness, zoom, etc
#https://towardsdatascience.com/exploring-image-data-augmentation-with-keras-and-tensorflow-a8162d89b844
train_dataGenerator = ImageDataGenerator(

    #https://github.com/Arsey/keras-transfer-learning-for-oxford102/issues/1 (this is the reason why we rescale to 1./255)
    rescale=1./255,

    #during training the image can roate up to 30 degrees (you can change this value if you want up to 360 i think)
    rotation_range=30,

    #self explainatory, allows more "data images" with a limited set because we can move the image in all 4 directions, look at link below for examples
    #https://stackoverflow.com/questions/62484597/understanding-width-shift-range-and-height-shift-range-arguments-in-kerass
    width_shift_range=0.2,
    height_shift_range=0.2,

    #similar to width/height shift range but it distorts the image instead of moving it
    #https://stackoverflow.com/questions/57301330/what-exactly-the-shear-do-in-imagedatagenerator-of-keras
    shear_range=0.2,

    #magnification range
    zoom_range=0.2,

    #it can flip the image
    horizontal_flip=True,

    #its the method used to fill the empty pixals(like for example we shift the image a bit to the right we have empty pixels on the left) although I don't get how it works
    fill_mode='nearest')

#we don't need any otherthing besides rescaling bc its the testing data set
test_datagen = ImageDataGenerator(rescale=1./255)

NameError: name 'ImageDataGenerator' is not defined

In [ ]:
#theres a error bc its unable to find the file btw (its saying something abt a naming error but I'm really hoping that adding images to our training data and testing data will fix this)
train_generator = train_dataGenerator.flow_from_directory(

    #its the path to the training images
    trainImages,

    #it has to be (224,224) because resnet50 uses 244,244
    target_size=(224, 224),

    #the number of images for each training run (this case its 25 but you can change depending on choice)
    batch_size=25,

    #we are classifiying based on true or false so we can use binary as our classifiction mode (0 for ai, 1 for human or you can swap it)
    class_mode='binary')

#btw .flow_from_directory has 3 requirements to work, 1: the root directory must have 2 folders(1 for training, 1 for testing), 2: the training folder should have a amount of sub directoies depending on what we are training on (our case its ai/human so two sub dierctories)
#3: training folder should only contain the test images
#https://studymachinelearning.com/keras-imagedatagenerator-with-flow_from_directory/#:~:text=The%20flow_from_directory()%20method%20takes,are%20using%20flow_from_directory()%20method.
test_generator = test_datagen.flow_from_directory(

    #path to test images
    testImages,

    #the rest is the same you get it (i just changed batch size to 50)
    target_size=(224, 224),
    batch_size=50,
    class_mode='binary')


NameError: name 'train_dataGenerator' is not defined

In [ ]:
#imagenet is us using pre-trained weigths from ImageNet(which are created using other datasets)
#IMPORTANT, note that I think google colab has accecss to the ImageNet database but I don't myself (im requesting access still)
#include_top=flase means that we remove the top layer of the model aka it allows us to use our own inputs for the model as well as controlling the output while being able to use the already exisiting weights from ImageNet.
#the input_shape is just (width, height, and rgb(3 means all 3 colors))
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
#it means that the ResNet50 model cant be modified when we train our own model(meaning the layers that resnet50 uses wont be changed)
base_model.trainable = False

In [ ]:
#BTW THIS IS WHERE OUR OWN MODEL IS MADE, each line is a new layer, ex: base_model, flatten, dense, and the others its why we rename sequential into 'model'

#also to note the Sequential is the type of model we use
#link for a bit of explainition on sequential model    https://www.tensorflow.org/guide/keras/sequential_model
model = Sequential([

    #the pretrained model
    base_model,

    #layers
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),

    #it classifiys the output into either 0 or 1 (final layer which is the classification one)
    Dense(1, activation='sigmoid')
])

In [ ]:
#compile model, meaning we find the values we need for science fair
#also we need to compile model in order for it to convert its data into things we can understand
#chat gpt the things im getting lazy
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
#summary, early stopping stops the model if there is no improvement, monitor is self explainatory(it looks for only 'val_loss', patience is the number of epochs that have to be the same,
# and restore best weights mean that before it stops it picks the best layer)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [ ]:
history = model.fit(

    #error bc in previous code it is doesn't exist atm (please i need help with this line train_generator = train_dataGenerator.flow_from_directory(  its actually ruining the rest of the code).
    train_generator,

    #// is the divide operator
    #and look here for epoch explaination https://www.geeksforgeeks.org/epoch-in-machine-learning/
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size,
    callbacks=[early_stopping])

NameError: name 'train_generator' is not defined

In [ ]:
#it just saves the model
model.save('ScienceFair.keras')

In [ ]:
#none of this works rn bc of the same reasons(test_generator file is not made)
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")

NameError: name 'test_generator' is not defined

In [ ]:
! pip install kaggle

In [ ]:
! chmod 600 ~/ .kaggle/kaggle.json

In [ ]:
#import kagglehub

## Download latest version
#path = kagglehub.dataset_download("mylesoneill/tagged-anime-illustrations")

#print("Path to dataset files:", path)